In [1]:
import numpy as np
import pandas as pd
import math
import datetime as dt

from datetime import datetime, date, time, timedelta

In [2]:
def fill_missing_index(df_list):
    df = df_list
    h = timedelta(hours = 1)
    return_list = []

    # 시작과 끝 검수
    for data in df:
        if(data['일시'][0].month != 5 or
           data['일시'][0].day != 1 or
           data['일시'][0].hour != 0):
            print('check head of data')

        elif(data['일시'][len(data) - 1].month != 10 or 
             data['일시'][len(data) - 1].day != 31 or 
             data['일시'][len(data) - 1].hour != 23):
            print('check tail of data')

    for data in df:
        missing_list = [] # missing_list는 결측된 시점들의 리스트

        if(len(data) != 4416):
            for index in range(len(data) - 1):
                diff = (data['일시'][index + 1] - data['일시'][index])/h
                diff = int(diff)

                if(diff != 1):
                    if(diff == 2):
                        missing = data['일시'][index] + h
                        print('missing datetime is', missing)
                        missing_list.append([index, diff])
                        print(' ')

                    else:
                        for d in range(diff - 1):
                            missing = data['일시'][index] + (d + 1)*h
                            print('missing datetime is', missing)
                        print(' ')
                        missing_list.append([index, diff])

            print('missing list is', missing_list)

            # 빈 날짜 채워넣기
            for missing in missing_list:
                if (missing[1] != 1):
                    empty = pd.DataFrame(index = range(missing[1] - 1), columns = data.columns)
                    for i in range(missing[1] - 1):
                        empty['일시'][i] = data['일시'][missing[0]] + (i + 1)*h
                    #print('empty is\n', empty)
                    temp1 = data[:missing[0] + 1]
                    temp2 = data[missing[0] + 1:]
                    data = pd.concat([temp1, empty, temp2], axis = 0, ignore_index = True)
                    
                    missing[0] = missing[0] + 1

                    if (missing_list.index(missing) < len(missing_list)):
                        for j in missing_list[missing_list.index(missing) + 1:]:
                            j[0] = j[0] + (missing[1] - 1)

                    #print('missing list is', missing_list, missing_list.index(missing))
                
            return_list.append([data, missing_list])
        else:
            return_list.append([data, missing_list])

    return return_list

In [3]:
def find_overtime(y_list):
    d1 = timedelta(days = 1)

    for year in y_list:
        y = year['일시'][0].year
        firstday = year['일시'][0]
        print('----------', y, '----------')

        # 첫날 검증
        if(firstday.year != y or
           firstday.month != 1 or
           firstday.day != 1 or
           firstday.hour != 0):
            print('check head of year', y)
        
        if(y/4 == 0):
            day_list = []
            for i in range(366):
                day_list.append(firstday + d1*i)
                pass

            for day in day_list:
                num = 0

                for row in range(len(year)):
                    if(year['일시'][row].month == day.month and
                      year['일시'][row].day == day.day):
                        num = num + 1
                        pass
                    pass

                if(num == 24):
                    pass
                else:
                    print('error in', day)
                    pass
                pass
            pass

        else:
            day_list = []
            for i in range(365):
                day_list.append(firstday + d1*i)
                pass

            for day in day_list:
                num = 0

                for row in range(len(year)):
                    if(year['일시'][row].month == day.month and
                      year['일시'][row].day == day.day):
                        num = num + 1
                        pass
                    pass

                if(num == 24):
                    pass
                elif(num > 24):
                    print('over in', day)
                    pass
                else:
                    print('missing in', day)
                    pass
                pass
            pass
        print('')

In [4]:
# 선형보간

def df_interpolate(list):
    return_list = []
    for data in list: # data[0]은 데이터셋, data[1]은 missing_list
        df = data.interpolate(method = 'linear')
        return_list.append(df)
    
    return return_list

In [5]:
def make_log_percip(df_list):
    scaled_list = []

    for df in df_list:
        log_list = []
        for item in df['강수량']:
            log_list.append(math.log(item + 1.0))
        
        df['log_percip'] = log_list
        df = df.drop('강수량', axis = 1)
        scaled_list.append(df)

    return scaled_list

In [6]:
def sep_from5to10(df_list): # 총 184일, 4416행
    df = df_list
    sep_list = []

    for data in df:
        drop_list = []
        for i in range(len(data)):
            if (data['일시'][i].month < 5 or data['일시'][i].month > 10):
                drop_list.append(i)
        data = data.drop(drop_list, axis = 0)
        data = data.reset_index(drop = True)
        sep_list.append(data)

    return sep_list

In [7]:
def check_date(list):
    first_day = dt.datetime(2021, 5, 1)
    last_day = dt.datetime(2021, 10, 31)

    h = timedelta(hours = 1)
    n = int((last_day - first_day)/h)

    for data in list:
            t = dt.datetime(year = data['일시'][0].year, month = 5, day = 1, hour = 0)
            time_list = []
    
            for i in range(n):
                time_list.append(t + i*h)
                
            for i in range(n):
                if(time_list[i].month == data['일시'][i].month or
                   time_list[i].day == data['일시'][i].day or
                   time_list[i].hour == data['일시'][i].hour ):
                    continue
                else:
                    print('error in year', time_list[i])

            print('no error in year', data['일시'][i].year)

In [8]:
def make_past_percip(p_list, n):
    return_list = []
    r = np.arange(n + 1, len(p_list))
    for i in range(n):
        return_list.append(0)
    for item in p_list[:len(p_list) - n]:
        return_list.append(item)

    return return_list

In [9]:
# n시간 전까지의 강수량을 추가
def make_include_past(data_set, n):
    p_list = []
    for item in data_set['log_percip']:
        p_list.append(item)

    before_list = []
    return_data = pd.DataFrame()
    return_data = data_set

    for i in range(n):
        a = make_past_percip(p_list, i + 1)
        before_list.append(a)
    
    for i in range(n):
        return_data['{}시간전'.format(i + 1)] = before_list[i]

    return return_data

In [10]:
def make_sequence(train, target, seq_length, elapsed_time):
    xs = []
    ys = []

    for j in range(len(train) - (seq_length + elapsed_time -1)):
        x = train[j : (j + seq_length)]
        y = target[j + (seq_length + elapsed_time - 1)]
        xs.append(x)
        ys.append(y)

    return np.array(xs), np.array(ys)

In [12]:
def make_seq_list(train_list, target_list, seq_length, elapsed_time, n):
    seq_list = []

    for i in range(n):
        seq_train, seq_target = make_sequence(train_list[i], target_list[i], seq_length, elapsed_time)
        seq_list.append([seq_train, seq_target])

    return seq_list

In [13]:
# # 강수량 구간별 개수 조사

# # 0, 0~1, 1~5, 5~10, 10~20, 20~
# count = [0, 0, 0, 0, 0, 0]

# for i in range(23):
#     for j in range(4416):
#         if(ASOS_sep[i]['강수량'][j] == 0):
#             count[0] = count[0] + 1
#         elif(ASOS_sep[i]['강수량'][j] <= 1):
#             count[1] = count[1] + 1
#         elif(ASOS_sep[i]['강수량'][j] <= 5):
#             count[2] = count[2] + 1
#         elif(ASOS_sep[i]['강수량'][j] <= 10):
#             count[3] = count[3] + 1
#         elif(ASOS_sep[i]['강수량'][j] <= 20):
#             count[4] = count[4] + 1
#         elif(ASOS_sep[i]['강수량'][j] > 20):
#             count[5] = count[5] + 1
#         else:
#             pass

# print(count)
# print(count[0] + count[1] + count[2] + count[3] + count[4] + count[5])
# print(count[0])
# print(count[1] + count[2] + count[3] + count[4] + count[5])
# print(4416*23)